2018/02/27Updated

[Darvas Box Traps Elusive Returns](https://www.investopedia.com/articles/trading/08/darvas.asp)

The rules can be explained so that modern tools like scanning software can identify trading candidates. To quantify the box, traders should look for stocks in which the difference between the high and the low price over **the past four weeks is less than 10% of the stock's high during that time**. As a formula, it can be written as:



```(100 * ((High – Low) / High)) < 10```

Traders can use a larger percentage to get more stocks on their potential buy lists. The buy should be taken at the market's open the morning after the stock closes outside the box by **at least half a point on a volume that is greater than the average 30-day volume**. The initial stop should be set a quarter point below the lowest price of the box. It should be raised as new boxes form, always a quarter point below the low.

---

box に入っている銘柄とは；
+ 過去4週間のHLの差をHで割ったものが0.1以下

box を抜けるとは，
+ 終値がボックスの外に，50％突き抜ける
+ 30日の出来高よりも多くの出来高がある

Entry
+ boxを抜けた次の日の朝


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from __future__ import division 

from quantopian.pipeline import Pipeline,CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters.morningstar import Q1500US,Q500US
from quantopian.pipeline.experimental import QTradableStocksUS

from quantopian.pipeline.filters import StaticAssets


from quantopian.pipeline.factors import DailyReturns, SimpleMovingAverage
import alphalens

dowlist = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DWDP', 'GE', 'GS',
 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE',
 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM']

class HighLowRatio(CustomFactor):
    # 前日までのデータでhighlowratio
    inputs = [ USEquityPricing.high, USEquityPricing.low]
    
    def compute(self, today, assets, out, high, low):
        highest = np.nanmax(high[:-1],axis=0)
        loweset = np.nanmin(low[:-1],axis=0)
        high_low_ratio = (highest - loweset)/ highest 
        
        out[:] = high_low_ratio #np.log(high_low_ratio) 

class PrevValue(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, value):
        out[:] = value[0]

class PrevMA(CustomFactor):
    def compute(self, today, assets, out, value):
        out[:] = np.nanmean(value[:-1], axis=0)
    
class ZSCORE(CustomFactor):
    def compute(self, today, assets, out, value):
        mean = np.nanmean(value, axis=0)
        stdev = np.nanstd(value, axis=0)
        out[:] = (value[-1] - mean) / stdev
    
def make_pipeline():
    pipe = Pipeline()
    HighLowRatio_4weeks = HighLowRatio(window_length=30)
    PrevDay_High = PrevValue(inputs=[USEquityPricing.high])
    PrevDay_Low = PrevValue(inputs=[USEquityPricing.low])
    PrevMA30_Vol = PrevMA(inputs=[USEquityPricing.volume], window_length=31)
    PrevMA30_Cls = PrevMA(inputs=[USEquityPricing.close], window_length=31)
    PrevMA30_MarketCap = PrevMA30_Cls * PrevMA30_Vol
    VolZScore30 = ZSCORE(inputs=[USEquityPricing.volume], window_length=30)

    pipe.add(HighLowRatio_4weeks, 'HighLowRatio_4weeks')
    pipe.add(USEquityPricing.close.latest, 'close')
    pipe.add(USEquityPricing.volume.latest, 'volume')
    pipe.add(PrevDay_High, 'PrevDay_High')
    pipe.add(PrevDay_Low, 'PrevDay_Low')
    pipe.add(PrevMA30_Vol, 'PrevMA30_Vol')
    pipe.add(PrevMA30_MarketCap, 'PrevMA30_MarketCap')
    pipe.add(VolZScore30, 'VolZScore30')
    
    my_symbols = StaticAssets(symbols(dowlist))
    pipe.set_screen(my_symbols)
    return pipe

def make_pipeline2():
    pipe = Pipeline()
    HighLowRatio_4weeks = HighLowRatio(window_length=30)
    pipe.add(HighLowRatio_4weeks, 'HighLowRatio_4weeks')
    
    my_symbols = StaticAssets(symbols(dowlist))
    pipe.set_screen(my_symbols)
    return pipe
    

In [ ]:
start="2010-1-1"
end="2018-3-13"
results = run_pipeline(make_pipeline(), start_date= start, end_date=end, )
inthebox = results[results["HighLowRatio_4weeks"] < 0.1]
inthebox["close/PrevDay_High"] = inthebox["close"]/inthebox["PrevDay_High"] - 1
inthebox["close/PrevDay_Low"] = inthebox["close"]/inthebox["PrevDay_Low"] - 1 


In [ ]:
symbol = "CVX"
idx = pd.IndexSlice
df = results.loc[idx[:,[symbols(symbol)]],:]
# df = df[1000:1500]

fig = plt.figure()
ax1 = fig.add_subplot(211)
ax2 = ax1.twinx()
ax3 = fig.add_subplot(212)

#ax = df[["close", "HighLowRatio_4weeks",]][:].plot(secondary_y="close")
#ax.axhline(y=0.1, c='red')
#ax.axhline(y=0.05, c='red')

ax1.plot(df.close.values)
ax2.plot(df["HighLowRatio_4weeks"].values, c='red')
ax2.axhline(y=-2.0, c='green')
ax2.axhline(y=-3.0, c='green')

ax3.plot(df["VolZScore30"].values)



In [ ]:
results.head(10).index.get_level_values(1).unique()


In [ ]:
target = inthebox[(inthebox["close/PrevDay_Low"] < -0.20) & (inthebox["PrevMA30_Vol"] * 1.5 < inthebox["volume"])]

dates = target.index.get_level_values(0)

for date in dates:
    tickers = target.loc[date].index
    df = get_pricing(tickers,start_date=date, end_date=date+ timedelta(days=10), fields='price',)
    dayreturn = df.pct_change().cumsum()
    plt.plot(dayreturn.as_matrix())
    


In [ ]:
target = inthebox[(inthebox["close/PrevDay_Low"] < -0.20) & (inthebox["PrevMA30_Vol"] * 1.5 < inthebox["volume"])]
alldates = results.index.get_level_values(0).unique()


In [ ]:
hoge = dict()
for (d,s), row in target.iterrows():
    ## 次の日の午前中の値動き
    i = alldates.get_loc(d) + 1
    nextdate = alldates[i]
    df = get_pricing(s, start_date=nextdate, end_date=nextdate, frequency='minute', fields='price')
    minutereturn = df[:90].pct_change()
    hoge[(d,s)] = minutereturn.as_matrix()
    plt.plot(minutereturn.cumsum().as_matrix())
    

In [ ]:
std = pd.DataFrame(hoge).dropna().describe().loc['std']
target['STD'] = std

In [ ]:
target.plot(x ="HighLowRatio_4weeks", y = "STD", kind='scatter')

In [ ]:
test = get_pricing(["AAPL", "F"])
test

In [ ]:
((test.high.rolling(window=20).max() - test.low.rolling(window=20).min())/test.high.rolling(window=20).max()).plot()

In [ ]:
aapl = get_pricing("aapl",start_date="2010-1-1", end_date="2018-1-1")
aapl["high low ratio"]  = (aapl.high.rolling(window=20).max() - aapl.low.rolling(window=20).min()) / aapl.high.rolling(window=20).max()

In [ ]:
aapl[["price", "high low ratio"]].plot(secondary_y = "price")


In [ ]:
start="2010-1-1"
end="2018-3-13"
results = run_pipeline(make_pipeline2(), start_date= start, end_date=end, )



In [ ]:
df = results.to_panel().HighLowRatio_4weeks

In [ ]:
dia = get_pricing('DIA', start_date=df.index[0], end_date=df.index[-1], fields='price')
df['DIA'] = dia

In [ ]:
df.loc[:"2010-06-30"].plot(legend=False, secondary_y ="DIA")
